# Continuous Bag of Words (CBOW) Model

In [96]:
corpus = [
    "Natural Language Processing is a fascinating field of study that has been evolving rapidly over the past few decades.",
    "Machine learning provides powerful tools for automating tasks and making predictions from data.",
    "Text data is often messy and unstructured, which makes it challenging to analyze and understand without the right tools.",
    "Deep learning models have shown remarkable success in understanding complex patterns in data, especially for tasks related to NLP.",
    "I love building machine learning models and experimenting with different techniques to improve their performance.",
    "Clean and properly preprocessed data is essential for building successful machine learning models that generalize well."
]

In [97]:
import pandas as pd
df = pd.read_csv("brown.csv")
corpus = df['tokenized_text'].tolist()
corpus = corpus[:10000]

train_data = corpus[:8000]
test_data = corpus[8000:]

corpus[:5]

['Furthermore , as an encouragement to revisionist thinking , it manifestly is fair to admit that any fraternity has a constitutional right to refuse to accept persons it dislikes .',
 'The Unitarian clergy were an exclusive club of cultivated gentlemen -- as the term was then understood in the Back Bay -- and Parker was definitely not a gentleman , either in theology or in manners .',
 'Ezra Stiles Gannett , an honorable representative of the sanhedrin , addressed himself frankly to the issue in 1845 , insisting that Parker should not be persecuted or calumniated and that in this republic no power to restrain him by force could exist .',
 "Even so , Gannett judiciously argued , the Association could legitimately decide that Parker `` should not be encouraged nor assisted in diffusing his opinions by those who differ from him in regard to their correctness '' .",
 'We today are not entitled to excoriate honest men who believed Parker to be downright pernicious and who barred their pulp

In [98]:
import re
import spacy
from pprint import pprint

nlp = spacy.load("en_core_web_sm")


def clean_text(documents: list[str]):
    cleaned_docs = []
    for doc in documents:
        doc_text = re.sub(r"[^\w\s]", "", doc.lower())
        doc_nlp = nlp(doc_text)
        filtered_text = [token.text for token in doc_nlp if not token.is_stop and token.text.strip()]
        cleaned_docs.append(filtered_text)

    return cleaned_docs

cleaned_corpus = clean_text(corpus[:5])
pprint(cleaned_corpus[:5])


[['furthermore',
  'encouragement',
  'revisionist',
  'thinking',
  'manifestly',
  'fair',
  'admit',
  'fraternity',
  'constitutional',
  'right',
  'refuse',
  'accept',
  'persons',
  'dislikes'],
 ['unitarian',
  'clergy',
  'exclusive',
  'club',
  'cultivated',
  'gentlemen',
  'term',
  'understood',
  'bay',
  'parker',
  'definitely',
  'gentleman',
  'theology',
  'manners'],
 ['ezra',
  'stiles',
  'gannett',
  'honorable',
  'representative',
  'sanhedrin',
  'addressed',
  'frankly',
  'issue',
  '1845',
  'insisting',
  'parker',
  'persecuted',
  'calumniated',
  'republic',
  'power',
  'restrain',
  'force',
  'exist'],
 ['gannett',
  'judiciously',
  'argued',
  'association',
  'legitimately',
  'decide',
  'parker',
  'encouraged',
  'assisted',
  'diffusing',
  'opinions',
  'differ',
  'regard',
  'correctness'],
 ['today',
  'entitled',
  'excoriate',
  'honest',
  'men',
  'believed',
  'parker',
  'downright',
  'pernicious',
  'barred',
  'pulpits',
  'dema

In [99]:
from collections import Counter

def build_vocab(corpus: list[str]):
    vocab = Counter(term for doc in corpus for term in doc)
    word_to_idx = {word: idx for idx, (word, _) in enumerate(vocab.items())}
    idx_to_word = {idx: word for idx, (word, _) in enumerate(vocab.items())}
    return word_to_idx, idx_to_word

word_to_idx, idx_to_word = build_vocab(cleaned_corpus[:5])
print(word_to_idx)
print(idx_to_word)
print(len(word_to_idx))

{'furthermore': 0, 'encouragement': 1, 'revisionist': 2, 'thinking': 3, 'manifestly': 4, 'fair': 5, 'admit': 6, 'fraternity': 7, 'constitutional': 8, 'right': 9, 'refuse': 10, 'accept': 11, 'persons': 12, 'dislikes': 13, 'unitarian': 14, 'clergy': 15, 'exclusive': 16, 'club': 17, 'cultivated': 18, 'gentlemen': 19, 'term': 20, 'understood': 21, 'bay': 22, 'parker': 23, 'definitely': 24, 'gentleman': 25, 'theology': 26, 'manners': 27, 'ezra': 28, 'stiles': 29, 'gannett': 30, 'honorable': 31, 'representative': 32, 'sanhedrin': 33, 'addressed': 34, 'frankly': 35, 'issue': 36, '1845': 37, 'insisting': 38, 'persecuted': 39, 'calumniated': 40, 'republic': 41, 'power': 42, 'restrain': 43, 'force': 44, 'exist': 45, 'judiciously': 46, 'argued': 47, 'association': 48, 'legitimately': 49, 'decide': 50, 'encouraged': 51, 'assisted': 52, 'diffusing': 53, 'opinions': 54, 'differ': 55, 'regard': 56, 'correctness': 57, 'today': 58, 'entitled': 59, 'excoriate': 60, 'honest': 61, 'men': 62, 'believed': 6

In [100]:
def create_context_target_pairs(corpus: list[str], window_size: int = 2):
    pairs = []
    for document in corpus:
        for idx, term in enumerate(document):
            start_idx = max(idx - window_size, 0)
            end_idx = min(idx + window_size + 1, len(document))
            pairs.append(([document[i] for i in range(start_idx, end_idx)], term))

    return pairs

pairs = create_context_target_pairs(cleaned_corpus)
pairs[:5]   

[(['furthermore', 'encouragement', 'revisionist'], 'furthermore'),
 (['furthermore', 'encouragement', 'revisionist', 'thinking'],
  'encouragement'),
 (['furthermore', 'encouragement', 'revisionist', 'thinking', 'manifestly'],
  'revisionist'),
 (['encouragement', 'revisionist', 'thinking', 'manifestly', 'fair'],
  'thinking'),
 (['revisionist', 'thinking', 'manifestly', 'fair', 'admit'], 'manifestly')]

In [101]:
def encode_pairs(pairs: list[str], word_to_idx: dict):
    encoded_pairs = []
    for context, target in pairs:
        context_idx = [word_to_idx[term] for term in context]
        target_idx = word_to_idx[target]
        encoded_pairs.append((context_idx, target_idx))

    return encoded_pairs

encoded_pairs = encode_pairs(pairs, word_to_idx)
encoded_pairs[:5]

[([0, 1, 2], 0),
 ([0, 1, 2, 3], 1),
 ([0, 1, 2, 3, 4], 2),
 ([1, 2, 3, 4, 5], 3),
 ([2, 3, 4, 5, 6], 4)]

In [102]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

def convert_to_dataset(encoded_pairs: list, batch_size: int):
    context_data, target_data = zip(*encoded_pairs)
    context_tensors = [torch.tensor(context, dtype=torch.long) for context in context_data]
    context_tensor = pad_sequence(context_tensors, batch_first=True)
    target_tensor = torch.tensor(target_data, dtype=torch.long)
    dataset = TensorDataset(context_tensor, target_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader

def create_dataset(corpus: list[str], batch_size: int, test_size: float = 0.2) -> tuple[DataLoader, DataLoader, int]:
    cleaned_corpus = clean_text(corpus)
    word_to_idx, idx_to_word = build_vocab(cleaned_corpus)
    pairs = create_context_target_pairs(cleaned_corpus)
    encoded_pairs = encode_pairs(pairs, word_to_idx)
    train_pairs, test_pairs = train_test_split(encoded_pairs, test_size=test_size, random_state=42)
    return convert_to_dataset(train_pairs, batch_size), convert_to_dataset(test_pairs, batch_size), len(word_to_idx)


`nn.Embedding(vocab_size, embedding_dim)`: This creates a lookup table for the embeddings. It takes two arguments:
- `vocab_size`: The size of the vocabulary (number of unique words in the corpus).
- `embedding_dim`: The dimension of the embeddings.

When you pass a tensor of indices to the `Embedding` layer, it looks up the embeddings for each index and returns a tensor of shape `(batch_size, sequence_length, embedding_dim)`.

in the forward pass, the mean of the embeddings is taken and then passed to the linear layer. this is necessary as mean is a good choice for cbow model because:
- Stability: Averaging smooths out noise and produces more stable representations.
- Invariance to Context Size: Whether you have 2 or 10 context words, you get an embedding of the same dimension and approximate magnitude.
- Computational Efficiency: It's simple and fast to compute.
- Theoretical Interpretation: It can be interpreted as estimating the expected context embedding for a given target word.


In [107]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context):
        embedded = self.embeddings(context)
        embedded = embedded.mean(dim=1)
        out = self.linear(embedded)
        return out


def train_model(model: nn.Module, train_loader: DataLoader, test_loader: DataLoader, epochs: int, learning_rate: float):
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    losses = []

    for epoch in tqdm(range(epochs), desc="Training", total=epochs):
        total_loss = 0
        model.train()
        for X_train, y_train in train_loader:
            y_pred = model(X_train)
            print(y_pred)
            print(f"y_pred.shape: {y_pred.shape}")
            print(y_train)
            print(f"y_train.shape: {y_train.shape}")
            loss = loss_function(y_pred, y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        losses.append(total_loss)

        if epoch % 10 == 0:
            print(f"Epoch {epoch} | loss: {total_loss}")

    return losses

train_loader, test_loader, vocab_size = create_dataset(corpus[:100], 32)
print(vocab_size)
embedding_dim = 10
model = CBOWModel(vocab_size=vocab_size, embedding_dim=embedding_dim)
losses = train_model(model, train_loader, test_loader, epochs=1, learning_rate=0.01)
losses

902


Training: 100%|██████████| 1/1 [00:00<00:00, 10.92it/s]

tensor([[ 0.3595, -0.1954,  0.1470,  ...,  0.3148,  0.1500,  0.3212],
        [ 0.2137, -0.1259,  0.3340,  ...,  0.2505,  0.0685,  0.4523],
        [ 0.2156,  0.4084, -0.4595,  ...,  0.3298,  0.1574, -0.2721],
        ...,
        [-0.2099,  0.2378,  0.1038,  ...,  0.1631, -0.1467, -0.3970],
        [ 0.6508,  0.3548,  0.0158,  ...,  0.7627,  0.3785, -0.3861],
        [ 0.7009,  0.4172,  0.1575,  ...,  0.7839,  0.1812, -0.0566]],
       grad_fn=<AddmmBackward0>)
y_pred.shape: torch.Size([32, 902])
tensor([547,  88, 291,  78, 830, 186, 818, 319, 571, 888, 516, 835, 535,  13,
         23, 186, 376, 538, 241, 564, 354, 769,  23, 747, 259, 667, 513, 114,
        252,  18, 605, 437])
y_train.shape: torch.Size([32])
tensor([[ 0.5864,  0.7046, -0.4820,  ...,  1.0216,  0.3166, -0.7296],
        [ 0.1198,  0.3302, -0.1496,  ...,  0.4059, -0.0496, -0.3484],
        [-0.1798, -0.0930,  0.6655,  ...,  0.0617, -0.1897, -0.0171],
        ...,
        [ 0.2400,  0.3594,  0.4369,  ...,  0.5559,  0.477

[206.53270196914673]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

def loss_curve(losses: list[float]):
    fig = px.line(x=range(len(losses)), y=losses, title='Loss Curve')
    fig.show()

loss_curve(losses)
